scaricare hdf view richiede acco gratuito per i download

+ link manuale prisma obsoleti? https://wiki.earthdata.nasa.gov/display/DAS/Toolkit+Downloads  

+ **prisma usa librerie hdf-eos5 v1.13, hdf-eos5 è ora a versione 1.16**  

+ esempio di mmap su hdf5(non hdf-eos5) con julia https://github.com/JuliaIO/HDF5.jl/blob/master/test/mmap.jl  

+ immagine docker per hdf eos lib https://hub.docker.com/r/hdfeos/hdfeos  

+ sdp toolkit: https://wiki.earthdata.nasa.gov/display/DAS/SDP+Toolkit+and+HDF-EOS+EOSDIS++Core+System+Project  

+ tutorial (in c) https://portal.hdfgroup.org/display/HDF5/Learning+the+Basics

+ hdfview: tool da https://support.hdfgroup.org/ftp/HDF5/releases/HDF-JAVA/hdf-java-2.10.1/bin/ (plugin non sembra funzionare con versione più recente)
  plugin per hdf-eos da https://wiki.earthdata.nasa.gov/display/DAS/HDFView+HDF-EOS+Plug-in    
  guida installazione: https://newsroom.gsfc.nasa.gov/sdptoolkit/HDFView/README_HDFView_plugin_v2.10.html    
  
+ documentazione pkg hdf5 https://juliaio.github.io/HDF5.jl/stable/  

+ esempi manipulaz hdf5 in julia https://www.juliabloggers.com/hdf5-in-julia/  

+ pkg R per importare dati prisma l1/l2 https://lbusett.github.io/prismaread/  
+ https://portal.hdfgroup.org/display/support/HDF5%201.12.0

In [2]:
# docu package hdf5 per julia https://juliaio.github.io/HDF5.jl/stable/
# using Pkg
# pkg.add("HDF5")
using HDF5
using DataFrames

#errori posson lasciare aperti file-> file inseriti in struct dinamica per poi esser facilmente chiusi tutti
openfiles = Dict()
counter = 0
# funzione che chiude tutti i file aperti
function closeall(dict)
    n = 0
    for (key, value) in dict
        close(pop!(dict, key))
        n=n+1;
    end
    counter = 0
    println("chiuso $n file")
end

closeall (generic function with 1 method)

apertura file con h5open(filename::string, mode::string) -> file   
mode:  

| mode | Meaning |
|---|---|
|"r"|read-only|
|"r+"|read-write, preserving any existing contents|
|"cw"|read-write, create file if not existing, preserve existing contents|
|"w"|read-write, destroying any existing contents (if any)|

In [5]:
testfilename = "test.h5"
fid = h5open(testfilename, "w") # crea file hdf5 vuoto (o lo apre e basta se già esiste)

close(fid)


In [6]:
close(fid)
rm(testfilename) #deleta file di prova

In [14]:
mutable struct lnode
    name::String
    leftmost::Union{lnode,Nothing}
    parent::Union{lnode,Nothing}
    rightsib::Union{lnode,Nothing}
    lnode(v::String) = (x = new(); x.leftmost = nothing; x.rightsib=nothing;x.parent=nothing; x.name=v; x)
end
# per lista dopiam collegata basta aggiungere un campo parent::lnode 

mutable struct treeList
    first::lnode
    List(root::lnode) = List(nodo)    
end

#struttura albero leftmost child/right sibling per alberi non binari

In [37]:
function getrightmost(parent::lnode)
    temp = parent
    if parent.leftmost != nothing
        temp = parent.leftmost
        while (temp.rightsib != nothing)
            temp = temp.rightsib
        end
        temp
    else
        throw("input has no parent")        
    end
end

function addchild!(parent::lnode, name::String)
    println("init addchild $name")
    new = lnode(name)
    new.parent = parent
    if(parent.leftmost!=nothing)
        temp = getrightmost(parent)
        temp.rightsib = new
        new
    else
        parent.leftmost = new
        new
    end
end

    

function viewfileTREE(x)#prende file, ritorna radice albero generato
    println("call viewfile") 
    myRoot = lnode("HDF5 FILE")
    nomi = names(x)
    checkAtts(x,myRoot)
    for n in nomi
        mychild = addchild!(myRoot,n)
        i = x[n]
        myView(i,mychild)
    end
    println("fine view file tree")
    myRoot 
    
end
    
    
function myView(me::HDF5.HDF5Attributes, mynode::lnode)#attributi
        #attributi non hanno figli
    println("view su attributo")
end

function myView(me::HDF5.HDF5Group, mynode::lnode)
    println("view su gruppo")
    #vediamo vediamo figli
    nomi = names(me)
    for n in nomi#per ogni sotto oggetto
        mychild = addchild!(mynode,n)#lo aggiunge all albero
        i = me[n]
        myView(i,mychild)#lancia la view su di esso
    end
    #controlliamo attributi
    checkAtts(me,mynode)
end

function myView(me::HDF5.HDF5Dataset, mynode::lnode)
    #data non ha figli
    checkAtts(me,mynode)
end

function checkAtts(x,mynode::lnode)
    attributi = attrs(x)
    natts = names(attributi)
    println("cerco attributi di $(mynode.name)")
    if isempty(natts)
        println("niente attributi")
    else 
        #println("ho trovato attributi $()")
        count = 0
        for attributo in natts
            addchild!(mynode, attributo)
            count=count+1
        end
        println("trovato $count atributi")
    end
end

function checkHeader()
    #todo
end
        

checkHeader (generic function with 1 method)

apre file
1. trova attributi file
2. per ogni attributo
    1. stampa nome, tipo, contenuto
2. trova sotto oggetti
3. per ogni sotto oggetto
    1. stampa nome, tipo
    2. trova attributi
    3. per ogni attrib
        1. stampa nome, tipo, contenuto
    5. se l'oggetto è un dataset, ne stampa le dimensioni ed il tipo di dati
    4. trova sotto oggetti (sse oggetto è gruppo, dataset non han sotto oggetti)    
    5. per ogni sotto oggetto
        1. vedi 4.

In [17]:
function viewfile(x)
    println("call viewfile")
   #=for i in x 
        if typeof(i)==HDF5Group
            viewgroup(i)
        else
            println(typeof(i))
        end
    end=#
    viewgroup(x)
    #dump(x)
end
    
function viewgroup(g)
    println("call viewgroup on $g")
    #nome
    #println(g)
    
    # attributi
    attributi = attrs(g)
    if isempty(names(attributi))
        println("niente attributi")
    else        
        viewattributes(attributi)
    end
    #attrs(obj) ritorna attributi oggetto
    nomi = names(g)
    # names(x::Union{HDF5Group,HDF5File}) ritorna array di nomi dei membri di obj
    println("nomi: $nomi")
    #view su oggetti contenuti
    for nome in nomi
        i = g[nome]
        ti = typeof(i)
        println("trovato sotto oggetto di nome: $nome e tipo $ti")
        if ti==HDF5Group            
            viewgroup(i)
        elseif ti==HDF5Dataset
            viewdata(i)
        else
            println(ti)
        end
    end
end

function viewdata(d)
    println("call viewdata on $d")
    
    #attributi
    attributi = attrs(d)
    if isempty(names(attributi))
        println("niente attributi")
    else        
        viewattributes(attributi)
    end
    
    # nome
    #println(d)
    #1 dataset mappabile?
    if ismmappable(d)
        println("dataset mmappabile")
    else
        println("dataset non mmappabile")
    end
    #mappatura dataset con readmmap() aumenta rpestazioni per letture numerose di un ds
    
    # datatype
    println("tipo dati: $(typeof(read(d)))")
    
    # dataspace (dimensioni )
    println("dimensioni: $(size(d))")
end 

function viewattributes(atts::HDF5.HDF5Attributes)
    println("call viewattributes on $atts")
    nomi = names(atts)
    for nome in nomi
        content = read(atts, nome)
        println("attributo di nome: $nome ; tipo: $(typeof(content))")
        println("contenuto: $content")
    end
end

viewattributes (generic function with 1 method)

In [ ]:
function printTree(root::lnode)
   #TODO 
end

In [51]:
function treeToDf(parent::Union{lnode})
    println("treetoDf on root")
    treeToDf(parent,0,0)
end

function treeToDf(p::lnode, pid, counter)#prende albero, ritorna df
    println("tree to df on: $(p.name)")
    println("figli? $(p.leftmost)")
    println("fratelli? $(p.rightsib)")
    
    if pid==0
            pid=nothing#per root
    end
    #si aggiunge
    myid=counter
    counter = counter+1
    @show mydf = DataFrame(id=myid,name=p.name,parent=pid)
    
    #lancia view su figlio        
    if p.leftmost!=nothing
        tempdf, counter = treeToDf(p.leftmost,myid,counter)
        append!(mydf,tempdf)
    end
    
    #lancia view sul fratello
    if p.rightsib!=nothing
        tempdf, counter = treeToDf(p.leftmost,pid,counter)
        append!(mydf,tempdf)
    end
    
    #ritorna df e counter
    mydf, counter
end

treeToDf (generic function with 4 methods)

In [52]:
#apro in lettura un file sample preso da hdfviewer, faccio qualcosa, chiudo file
#inputfile = "OMI-Aura_.he5"
inputfile = "data/PRS_L2D_STD_20200627102334_20200627102339_0001.he5"
println("ishdf5 ? $(ishdf5(inputfile))") #ishdf5 prende stringa (nome file) ritorna se file è hdf5
file2 = h5open(inputfile, "r")
openfiles["file $counter"]=file2
counter = counter+1
#foo(file2)
mytree = viewfileTREE(file2)
mydf = treeToDf(mytree)
closeall(openfiles)

ishdf5 ? true
call viewfile
cerco attributi di HDF5 FILE
init addchild Acquisition_Purpose
init addchild Acquisition_Size
init addchild Acquisition_Station
init addchild Acquisition_Type
init addchild Atm_LutGeomInfo_RelativeAzimuth
init addchild Atm_LutGeomInfo_SunZenith
init addchild Atm_LutGeomInfo_ViewZenith
init addchild Atm_Lut_version
init addchild Atmo_RTM_info
init addchild Atmo_profile_info
init addchild Aux_SunEarthDistance
init addchild Aux_SunIrradiance
init addchild CNM_L2_BINNING
init addchild CNM_L2_BIN_ON
init addchild CNM_L2_BSEL_ON
init addchild CNM_L2_HGRP
init addchild CNM_PAN_ACQ
init addchild CNM_SWIR_ACQ
init addchild CNM_SWIR_SELECT
init addchild CNM_VNIR_ACQ
init addchild CNM_VNIR_SELECT
init addchild Cloudy_pixels_percentage
init addchild DEM_info
init addchild Epsg_Code
init addchild Exit_Code
init addchild Frame_Type
init addchild GCP_info
init addchild ISF_ID_Start
init addchild Image_ID
init addchild Integration_Time
init addchild L1_Processor_Version
ini

init addchild Frame_Missing
cerco attributi di Frame_Missing
niente attributi
cerco attributi di Header
niente attributi
cerco attributi di Info
niente attributi
init addchild KDP_AUX
view su gruppo
init addchild Cw_Swir_Matrix
cerco attributi di Cw_Swir_Matrix
niente attributi
init addchild Cw_Vnir_Matrix
cerco attributi di Cw_Vnir_Matrix
niente attributi
init addchild Fwhm_Swir_Matrix
cerco attributi di Fwhm_Swir_Matrix
niente attributi
init addchild Fwhm_Vnir_Matrix
cerco attributi di Fwhm_Vnir_Matrix
niente attributi
init addchild LOS_Pan
cerco attributi di LOS_Pan
niente attributi
init addchild LOS_Swir
cerco attributi di LOS_Swir
niente attributi
init addchild LOS_Vnir
cerco attributi di LOS_Vnir
niente attributi
cerco attributi di KDP_AUX
niente attributi
fine view file tree
treetoDf on root
tree to df on: HDF5 FILE


figli? lnode("Acquisition_Purpose", nothing, lnode("HDF5 FILE", lnode(#= circular reference @-2 =#), nothing, nothing), lnode("Acquisition_Size", nothing, lnode("HDF5 FILE", lnode(#= circular reference @-3 =#), nothing, nothing), lnode("Acquisition_Station", nothing, lnode("HDF5 FILE", lnode(#= circular reference @-4 =#), nothing, nothing), lnode("Acquisition_Type", nothing, lnode("HDF5 FILE", lnode(#= circular reference @-5 =#), nothing, nothing), lnode("Atm_LutGeomInfo_RelativeAzimuth", nothing, lnode("HDF5 FILE", lnode(#= circular reference @-6 =#), nothing, nothing), lnode("Atm_LutGeomInfo_SunZenith", nothing, lnode("HDF5 FILE", lnode(#= circular reference @-7 =#), nothing, nothing), lnode("Atm_LutGeomInfo_ViewZenith", nothing, lnode("HDF5 FILE", lnode(#= circular reference @-8 =#), nothing, nothing), lnode("Atm_Lut_version", nothing, lnode("HDF5 FILE", lnode(#= circular reference @-9 =#), nothing, nothing), lnode("Atmo_RTM_info", nothing, lnode("HDF5 FILE", lnode(#= circular refer

mydf = DataFrame(id = myid, name = p.name, parent = pid) = 1×3 DataFrame
│ Row │ id    │ name      │ parent  │
│     │ Int64 │ String    │ Nothing │
├─────┼───────┼───────────┼─────────┤
│ 1   │ 0     │ HDF5 FILE │         │
tree to df on: Acquisition_Purpose
figli? nothing


fratelli? lnode("Acquisition_Size", nothing, lnode("HDF5 FILE", lnode("Acquisition_Purpose", nothing, lnode(#= circular reference @-2 =#), lnode(#= circular reference @-3 =#)), nothing, nothing), lnode("Acquisition_Station", nothing, lnode("HDF5 FILE", lnode("Acquisition_Purpose", nothing, lnode(#= circular reference @-2 =#), lnode(#= circular reference @-4 =#)), nothing, nothing), lnode("Acquisition_Type", nothing, lnode("HDF5 FILE", lnode("Acquisition_Purpose", nothing, lnode(#= circular reference @-2 =#), lnode(#= circular reference @-5 =#)), nothing, nothing), lnode("Atm_LutGeomInfo_RelativeAzimuth", nothing, lnode("HDF5 FILE", lnode("Acquisition_Purpose", nothing, lnode(#= circular reference @-2 =#), lnode(#= circular reference @-6 =#)), nothing, nothing), lnode("Atm_LutGeomInfo_SunZenith", nothing, lnode("HDF5 FILE", lnode("Acquisition_Purpose", nothing, lnode(#= circular reference @-2 =#), lnode(#= circular reference @-7 =#)), nothing, nothing), lnode("Atm_LutGeomInfo_ViewZenith

LoadError: type Nothing has no field leftmost

In [43]:
closeall(openfiles)

chiuso 4 file


In [19]:
inputfile = "data/PRS_L2D_STD_20200627102334_20200627102339_0001.he5"
println("ishdf5 ? $(ishdf5(inputfile))") #ishdf5 prende stringa (nome file) ritorna se file è hdf5
file2 = h5open(inputfile, "r")
openfiles["file $counter"]=file2
counter = counter+1
#foo(file2)
viewfile(file2)
closeall(openfiles)

Dict{Any,Any}()

## R
https://shekeine.github.io/modis/2014/08/27/HDF_to_multi-band_geotiff

https://hakimabdi.com/blog/working-with-hdf-eos-files-in-r-convert-project-and-mosaicù

installa gdalUtils package
installa gdalUtilities package

In [11]:
using DataFrames
df = DataFrame(id = 1, name = "HDF5FILE", parent = missing)

,id,name,parent
,Int64,String,Missing
1,1,HDF5FILE,missing


In [6]:
inputfile = "HDF5_test.h5"
println("ishdf5 ? $(ishdf5(inputfile))") #ishdf5 prende stringa (nome file) ritorna se file è hdf5
file2 = h5open(inputfile, "r")
openfiles["file $counter"]=file2
names(file2)

ishdf5 ? true


5-element Array{String,1}:
 "A note"
 "Notes"
 "arrays"
 "datatypes"
 "images"

In [8]:
closeall(openfiles)

chiuso 1 file


In [18]:
c=0
c!=nothing

true

In [19]:
myarr= [1,2,3]
append!(myarr, 4)

4-element Array{Int64,1}:
 1
 2
 3
 4

In [21]:
function tryDFLoop(names)
    df = nothing
    counter = 0
    for name in names
        tempdf = DataFrame(id = counter, nome = name)
        counter = counter+1
        if df!=nothing
            append!(df,tempdf)
        else
            df = tempdf
        end
    end
    df
end

tryDFLoop (generic function with 1 method)

In [23]:
nas = ["primo","secondo","terzo"]
tryDFLoop(nas)

,id,nome
,Int64,String
1,0,primo
2,1,secondo
3,2,terzo
